# EPA1361 - Model-Based Decision Making

## Multi-model analysis

This exercise uses a simple version of the [Lotka-Volterra predator-prey equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations) to show how the EMA Workbench can be used for a
multi-model analysis, in addition to typical parametric/structural uncertainties. This will let you test the connectors provided in the Workbench for Excel, NetLogo, and Vensim / PySD; we'll also use the models for the sensitivity analysis exercise in week 3.

**Assignment**
Using the three model files provided and the Python function below, define model objects for each implementation (Excel, NetLogo, Vensim/PySD, and Python), and test them using a single ensemble. Use 50 experiments sampled from the parameters below (so that each experiment will be executed for the 4 models, for a total of 200), and retrieve outputs for the _TIME_, _predators_, and _prey_ variables.
   * Excel and Vensim are only supported on Windows
   * Vensim requires the DSS version of Vensim
   * Netlogo supoprt depends on [jpype](http://jpype.readthedocs.io/en/latest/install.html) and [pynetlogo](https://pynetlogo.readthedocs.io/en/latest/). Also, if you don't have NetLogo installed, please get [NetLogo 6.3.0](https://ccl.northwestern.edu/netlogo/download.shtml)
   * for pysd, see [its documentation](http://pysd.readthedocs.io/en/master/installation.html)
   * If possible try to work with all model versions, but even 2 or 3 (pure python and something else should be sufficient).


|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |
|Final time	    |365	    |
|dt	    |0.25	    |

* Note that your EMA Workbench installation includes [example scripts](https://github.com/quaquel/EMAworkbench/tree/master/ema_workbench/examples) for the different connectors. The different model objects follow a similar syntax but will need to be slightly adjusted depending on the software (e.g. to specify the NetLogo run length or the sheet name in Excel).
  * This [tutorial](https://emaworkbench.readthedocs.io/en/latest/basic_tutorial.html) also shows a simple model in Python, Vensim and Excel connected to the workbench.

* These model objects can be used with a replication functionality (for instance to test the effect of stochastic uncertainty in a NetLogo model), which repeats a given experiment over multiple replications. You can use a single replication in this exercise as the models are not stochastic. By default, each outcome array will then have a shape of (# experiments, # replications, # time steps). Try adapting the outcome arrays so that they can be used with the _lines_ plotting function of the Workbench, and plot the results grouped by model.

* To check the graphical results, find the maximum absolute error of the time series you obtained for the _prey_ variable in the Excel, NetLogo, and Vensim/PySD models, relative to the Python function.

In [2]:
# Some imports you may need
import numpy as np
import matplotlib.pyplot as plt

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, perform_experiments, ema_logging, ScalarOutcome)

from ema_workbench.connectors.netlogo import NetLogoModel
from ema_workbench.connectors.excel import ExcelModel
from ema_workbench.connectors.pysd_connector import PysdModel

from ema_workbench.em_framework.samplers import LHSSampler
from ema_workbench.em_framework.salib_samplers import MorrisSampler, SobolSampler

from ema_workbench.analysis.plotting import lines, Density

C:\Users\muham\OneDrive - Delft University of Technology\Documents\Kuliah\S2\Q1.4\Model-based Decision Making\Assignments\venv\Lib\site-packages\ema_workbench\connectors\__init__.py:19: ImportWarning: vensim connector not available
  warnings.warn("vensim connector not available", ImportWarning)
C:\Users\muham\OneDrive - Delft University of Technology\Documents\Kuliah\S2\Q1.4\Model-based Decision Making\Assignments\venv\Lib\site-packages\ema_workbench\connectors\__init__.py:34: ImportWarning: simio connector not available
  warnings.warn("simio connector not available", ImportWarning)
C:\Users\muham\OneDrive - Delft University of Technology\Documents\Kuliah\S2\Q1.4\Model-based Decision Making\Assignments\venv\Lib\site-packages\ema_workbench\analysis\prim.py:32: ImportWarning: altair based interactive inspection not available
  warnings.warn("altair based interactive inspection not available", ImportWarning)


In [3]:
# Import the Python function
from model.pred_prey import PredPrey

In [4]:
# Define uncertainties and outcomes
model_uncertainties = [
    RealParameter('prey_birth_rate', 0.015, 0.035),
    RealParameter('predation_rate', 0.0005, 0.003),
    RealParameter('predator_efficiency', 0.001, 0.004),
    RealParameter('predator_loss_rate', 0.04, 0.08)
]

model_outcomes = [
    TimeSeriesOutcome('TIME'),
    TimeSeriesOutcome('predators'),
    TimeSeriesOutcome('prey')
]

# Instantiate Python Model
python_model = Model('PredPreyPython', function=PredPrey)

python_model.uncertainties = model_uncertainties
python_model.outcomes = model_outcomes

# Instantiate NetLogo Model
netlogo_model = NetLogoModel(name='PredPreyNetLogo', wd=r'./model', model_file='PredPrey.nlogo')

netlogo_model.setup_commands = ['setup']
netlogo_model.run_command = 'go'

netlogo_model.uncertainties = model_uncertainties
netlogo_model.outcomes = model_outcomes

# Define model objects for the different implementations

In [4]:
# Run the Python Model
if __name__ == "__main__":
    from ema_workbench import MultiprocessingEvaluator, ema_logging

    ema_logging.log_to_stderr(ema_logging.INFO)
    with MultiprocessingEvaluator(python_model, n_processes=4) as evaluator:
        experiments, outcomes = evaluator.perform_experiments(scenarios=50)


[MainProcess/INFO] pool started with 4 workers
[MainProcess/INFO] performing 50 scenarios * 1 policies * 1 model(s) = 50 experiments
100%|██████████████████████████████████████████| 50/50 [00:01<00:00, 35.78it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [5]:
experiments

,predation_rate,predator_efficiency,predator_loss_rate,prey_birth_rate,scenario,policy,model
0,0.001003,0.001526,0.063499,0.018367,0,None,PredPreyPython
1,0.002475,0.002738,0.068562,0.028117,1,None,PredPreyPython
2,0.001495,0.002950,0.056498,0.023558,2,None,PredPreyPython
3,0.000884,0.001235,0.070272,0.034189,3,None,PredPreyPython
4,0.002430,0.001079,0.075635,0.017904,4,None,PredPreyPython
5,0.002585,0.002442,0.059490,0.019565,5,None,PredPreyPython
6,0.001396,0.003851,0.053589,0.016906,6,None,PredPreyPython
7,0.000688,0.001618,0.049361,0.034416,7,None,PredPreyPython
8,0.001691,0.002017,0.071339,0.031193,8,None,PredPreyPython
9,0.002068,0.003108,0.045220,0.018921,9,None,PredPreyPython


In [6]:
# Run Model

from ema_workbench import perform_experiments

models = python_model
results = perform_experiments(models, scenarios=50)


100%|█████████████████████████████████████████| 50/50 [00:00<00:00, 567.56it/s]


In [9]:
experiments, outcomes = results
experiments

,predation_rate,predator_efficiency,predator_loss_rate,prey_birth_rate,scenario,policy,model
0,0.001046,0.003546,0.045709,0.020928,50,None,PredPreyPython
1,0.000797,0.003179,0.069889,0.030064,51,None,PredPreyPython
2,0.002867,0.002178,0.062921,0.027245,52,None,PredPreyPython
3,0.000984,0.001844,0.076255,0.021745,53,None,PredPreyPython
4,0.002632,0.002582,0.074821,0.020070,54,None,PredPreyPython
5,0.002707,0.003613,0.060403,0.033255,55,None,PredPreyPython
6,0.001862,0.003511,0.065766,0.016218,56,None,PredPreyPython
7,0.002024,0.003954,0.050874,0.022818,57,None,PredPreyPython
8,0.002466,0.002530,0.048458,0.015658,58,None,PredPreyPython
9,0.000689,0.003323,0.049926,0.021081,59,None,PredPreyPython
